实验MLP和LeNet在集中数据集训练下的模型收敛情况
​	

| mlp accuracy>=97%           | 200  | 100  | 50   | 10   | 200  | 100  | 50   | 10   |
| --------------------------- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| 0.01                        | 84   | 41   | 23   | 6    |      |      |      |      |
| 0.02                        | 43   | 21   | 11   | 3    |      |      |      |      |
| 0.04                        | 21   | 11   | 6    | 3    |      |      |      |      |
| **LeNet-5 accuracy>=98.8%** |      |      |      |      |      |      |      |      |
| 0.01                        | 132  | 66   | 29   | 11   |      |      |      |      |
| 0.02                        | 54   | 27   | 18   | 4    |      |      |      |      |
| 0.04                        | 34   | 13   | 11   | 6    |      |      |      |      |

​		

In [1]:
import mxnet as mx
from mxnet import gluon
from mxnet.gluon import nn
from mxnet import autograd as ag
from mxnet import ndarray as nd
from Algorithm.CNN import CNN_Model
from Algorithm.MLP import MLP
from Tools import utils

In [2]:
LeNet = CNN_Model('LeNet')
mlp = MLP()
ctx = utils.try_all_gpus()

In [3]:
LeNet.initialize(init=mx.initializer.Xavier(),ctx=ctx)
x = nd.random.uniform(low=0,high=255,shape=(1,1,28,28),ctx=ctx[0])
LeNet(x)
LeNet.save_parameters("LeNet.params")
mlp.initialize(init=mx.initializer.Xavier(),ctx=ctx)
x = x = nd.random.uniform(low=0,high=255,shape=(1,28,28),ctx=ctx[0])
mlp(x)
mlp.save_parameters("mlp.params")

mnist = mx.test_utils.get_mnist()
batch_size = 100
train_data = mx.io.NDArrayIter(mnist['train_data'], mnist['train_label'], batch_size, shuffle=True)
val_data = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], batch_size)

In [8]:
def validation(val_data, net):
    metric = mx.metric.Accuracy()
    val_data.reset()
    for batch in val_data:
        data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
        label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
        outputs = []
        for x in data:
            outputs.append(net(x))
        metric.update(label, outputs)
    name,acc =metric.get()
    return acc

def train(net, data, epoch, stop_acc):
    trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.02})
    metric = mx.metric.Accuracy()
    softmax_cross_entropy_loss = gluon.loss.SoftmaxCrossEntropyLoss()
    for i in range(epoch):
        train_data.reset()
        for batch in train_data:
            data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
            label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
            outputs = []
            with ag.record():
                for x, y in zip(data, label):
                    z = net(x)
                    loss = softmax_cross_entropy_loss(z, y)
                    loss.backward()
                    outputs.append(z)
            metric.update(label, outputs)
            trainer.step(batch.data[0].shape[0])
        name, acc = metric.get()
        metric.reset()
        print('training acc at epoch %d: %s=%f'%(i, name, acc))

def train_till_acc(net, stop_acc, lr, batch_size):
    train_data = mx.io.NDArrayIter(mnist['train_data'], mnist['train_label'], batch_size, shuffle=True)
    val_data = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], batch_size)
    trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
    softmax_cross_entropy_loss = gluon.loss.SoftmaxCrossEntropyLoss()
    i=0
    while(True):
        train_data.reset()
        for batch in train_data:
            data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
            label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
            outputs = []
            with ag.record():
                for x, y in zip(data, label):
                    z = net(x)
                    loss = softmax_cross_entropy_loss(z, y)
                    loss.backward()
                    outputs.append(z)
            trainer.step(batch.data[0].shape[0])
        acc = validation(val_data,net)
        i += 1
        if acc >= stop_acc:
            print('training acc at epoch %d: acc=%f'%(i, acc))
            break
        else:
            print('training acc at epoch %d: acc=%f'%(i, acc))

In [6]:
mlp = MLP()
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp, 0.97, 0.01, 200)

training acc at epoch 1: acc=0.815300
training acc at epoch 2: acc=0.871100
training acc at epoch 3: acc=0.888200
training acc at epoch 4: acc=0.898400
training acc at epoch 5: acc=0.904300
training acc at epoch 6: acc=0.909000
training acc at epoch 7: acc=0.913100
training acc at epoch 8: acc=0.913700
training acc at epoch 9: acc=0.918500
training acc at epoch 10: acc=0.921000
training acc at epoch 11: acc=0.925000
training acc at epoch 12: acc=0.925700
training acc at epoch 13: acc=0.926500
training acc at epoch 14: acc=0.929500
training acc at epoch 15: acc=0.931700
training acc at epoch 16: acc=0.934700
training acc at epoch 17: acc=0.935300
training acc at epoch 18: acc=0.937500
training acc at epoch 19: acc=0.938700
training acc at epoch 20: acc=0.939900
training acc at epoch 21: acc=0.940700
training acc at epoch 22: acc=0.942200
training acc at epoch 23: acc=0.943000
training acc at epoch 24: acc=0.944100
training acc at epoch 25: acc=0.944200
training acc at epoch 26: acc=0.94

In [7]:
mlp = MLP()
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp, 0.97, 0.01, 100)

training acc at epoch 1: acc=0.870300
training acc at epoch 2: acc=0.898500
training acc at epoch 3: acc=0.910400
training acc at epoch 4: acc=0.916100
training acc at epoch 5: acc=0.920400
training acc at epoch 6: acc=0.923200
training acc at epoch 7: acc=0.928800
training acc at epoch 8: acc=0.934200
training acc at epoch 9: acc=0.937000
training acc at epoch 10: acc=0.939400
training acc at epoch 11: acc=0.942200
training acc at epoch 12: acc=0.943500
training acc at epoch 13: acc=0.946400
training acc at epoch 14: acc=0.946900
training acc at epoch 15: acc=0.949600
training acc at epoch 16: acc=0.950400
training acc at epoch 17: acc=0.951600
training acc at epoch 18: acc=0.953100
training acc at epoch 19: acc=0.954500
training acc at epoch 20: acc=0.955600
training acc at epoch 21: acc=0.957900
training acc at epoch 22: acc=0.958400
training acc at epoch 23: acc=0.959900
training acc at epoch 24: acc=0.959400
training acc at epoch 25: acc=0.960200
training acc at epoch 26: acc=0.96

In [9]:
mlp = MLP()
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp, 0.97, 0.01, 50)

training acc at epoch 1: acc=0.896100
training acc at epoch 2: acc=0.910700
training acc at epoch 3: acc=0.925400
training acc at epoch 4: acc=0.932700
training acc at epoch 5: acc=0.939300
training acc at epoch 6: acc=0.944600
training acc at epoch 7: acc=0.946000
training acc at epoch 8: acc=0.949400
training acc at epoch 9: acc=0.952400
training acc at epoch 10: acc=0.955000
training acc at epoch 11: acc=0.957700
training acc at epoch 12: acc=0.960000
training acc at epoch 13: acc=0.961700
training acc at epoch 14: acc=0.963000
training acc at epoch 15: acc=0.963800
training acc at epoch 16: acc=0.965400
training acc at epoch 17: acc=0.966800
training acc at epoch 18: acc=0.967900
training acc at epoch 19: acc=0.968100
training acc at epoch 20: acc=0.968600
training acc at epoch 21: acc=0.969700
training acc at epoch 22: acc=0.969700
training acc at epoch 23: acc=0.971000


In [10]:
mlp = MLP()
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp, 0.97, 0.01, 10)

training acc at epoch 1: acc=0.937900
training acc at epoch 2: acc=0.953300
training acc at epoch 3: acc=0.954900
training acc at epoch 4: acc=0.964600
training acc at epoch 5: acc=0.968100
training acc at epoch 6: acc=0.972900


In [11]:
mlp = MLP()
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp, 0.97, 0.02, 200)

training acc at epoch 1: acc=0.871200
training acc at epoch 2: acc=0.898900
training acc at epoch 3: acc=0.909000
training acc at epoch 4: acc=0.915100
training acc at epoch 5: acc=0.920000
training acc at epoch 6: acc=0.925700
training acc at epoch 7: acc=0.930500
training acc at epoch 8: acc=0.934200
training acc at epoch 9: acc=0.935600
training acc at epoch 10: acc=0.939600
training acc at epoch 11: acc=0.939600
training acc at epoch 12: acc=0.943600
training acc at epoch 13: acc=0.945600
training acc at epoch 14: acc=0.947600
training acc at epoch 15: acc=0.946900
training acc at epoch 16: acc=0.949300
training acc at epoch 17: acc=0.952500
training acc at epoch 18: acc=0.953400
training acc at epoch 19: acc=0.954600
training acc at epoch 20: acc=0.955700
training acc at epoch 21: acc=0.956200
training acc at epoch 22: acc=0.958200
training acc at epoch 23: acc=0.958000
training acc at epoch 24: acc=0.959700
training acc at epoch 25: acc=0.960900
training acc at epoch 26: acc=0.96

In [12]:
mlp = MLP()
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp, 0.97, 0.02, 100)

training acc at epoch 1: acc=0.893800
training acc at epoch 2: acc=0.914600
training acc at epoch 3: acc=0.924600
training acc at epoch 4: acc=0.931200
training acc at epoch 5: acc=0.940800
training acc at epoch 6: acc=0.944300
training acc at epoch 7: acc=0.946800
training acc at epoch 8: acc=0.950800
training acc at epoch 9: acc=0.952800
training acc at epoch 10: acc=0.955600
training acc at epoch 11: acc=0.957500
training acc at epoch 12: acc=0.960200
training acc at epoch 13: acc=0.961600
training acc at epoch 14: acc=0.963100
training acc at epoch 15: acc=0.964300
training acc at epoch 16: acc=0.964500
training acc at epoch 17: acc=0.967300
training acc at epoch 18: acc=0.967300
training acc at epoch 19: acc=0.968400
training acc at epoch 20: acc=0.968800
training acc at epoch 21: acc=0.970000


In [13]:
mlp = MLP()
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp, 0.97, 0.02, 50)

training acc at epoch 1: acc=0.911000
training acc at epoch 2: acc=0.931300
training acc at epoch 3: acc=0.942800
training acc at epoch 4: acc=0.949300
training acc at epoch 5: acc=0.955400
training acc at epoch 6: acc=0.959600
training acc at epoch 7: acc=0.962400
training acc at epoch 8: acc=0.965600
training acc at epoch 9: acc=0.967500
training acc at epoch 10: acc=0.969900
training acc at epoch 11: acc=0.970300


In [14]:
mlp = MLP()
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp, 0.97, 0.02, 10)

training acc at epoch 1: acc=0.951500
training acc at epoch 2: acc=0.964400
training acc at epoch 3: acc=0.972500


In [15]:
mlp = MLP()
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp, 0.97, 0.04, 200)

training acc at epoch 1: acc=0.894500
training acc at epoch 2: acc=0.913800
training acc at epoch 3: acc=0.925000
training acc at epoch 4: acc=0.931600
training acc at epoch 5: acc=0.939900
training acc at epoch 6: acc=0.943200
training acc at epoch 7: acc=0.947500
training acc at epoch 8: acc=0.949700
training acc at epoch 9: acc=0.953500
training acc at epoch 10: acc=0.956500
training acc at epoch 11: acc=0.958600
training acc at epoch 12: acc=0.959200
training acc at epoch 13: acc=0.960600
training acc at epoch 14: acc=0.962800
training acc at epoch 15: acc=0.962400
training acc at epoch 16: acc=0.965800
training acc at epoch 17: acc=0.967000
training acc at epoch 18: acc=0.967700
training acc at epoch 19: acc=0.968400
training acc at epoch 20: acc=0.969500
training acc at epoch 21: acc=0.970700


In [16]:
mlp = MLP()
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp, 0.97, 0.04, 100)

training acc at epoch 1: acc=0.914400
training acc at epoch 2: acc=0.931300
training acc at epoch 3: acc=0.944000
training acc at epoch 4: acc=0.951500
training acc at epoch 5: acc=0.953500
training acc at epoch 6: acc=0.958900
training acc at epoch 7: acc=0.962000
training acc at epoch 8: acc=0.962500
training acc at epoch 9: acc=0.967200
training acc at epoch 10: acc=0.969200
training acc at epoch 11: acc=0.970800


In [17]:
mlp = MLP()
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp, 0.97, 0.04, 50)

training acc at epoch 1: acc=0.930200
training acc at epoch 2: acc=0.940200
training acc at epoch 3: acc=0.954900
training acc at epoch 4: acc=0.964700
training acc at epoch 5: acc=0.966000
training acc at epoch 6: acc=0.971400


In [18]:
mlp = MLP()
mlp.load_parameters("mlp.params",ctx=ctx)
train_till_acc(mlp, 0.97, 0.04, 10)

training acc at epoch 1: acc=0.961300
training acc at epoch 2: acc=0.969700
training acc at epoch 3: acc=0.974300


LeNet

In [22]:
LeNet = CNN_Model('LeNet')
LeNet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(LeNet, 0.988, 0.01, 200)

training acc at epoch 1: acc=0.837300
training acc at epoch 2: acc=0.918400
training acc at epoch 3: acc=0.931400
training acc at epoch 4: acc=0.945000
training acc at epoch 5: acc=0.949300
training acc at epoch 6: acc=0.958000
training acc at epoch 7: acc=0.960400
training acc at epoch 8: acc=0.964000
training acc at epoch 9: acc=0.967800
training acc at epoch 10: acc=0.968000
training acc at epoch 11: acc=0.968300
training acc at epoch 12: acc=0.974000
training acc at epoch 13: acc=0.971600
training acc at epoch 14: acc=0.972200
training acc at epoch 15: acc=0.973300
training acc at epoch 16: acc=0.974200
training acc at epoch 17: acc=0.975000
training acc at epoch 18: acc=0.976700
training acc at epoch 19: acc=0.977400
training acc at epoch 20: acc=0.979000
training acc at epoch 21: acc=0.980400
training acc at epoch 22: acc=0.978200
training acc at epoch 23: acc=0.980500
training acc at epoch 24: acc=0.981700
training acc at epoch 25: acc=0.980600
training acc at epoch 26: acc=0.98

In [23]:
LeNet = CNN_Model('LeNet')
LeNet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(LeNet, 0.988, 0.01, 100)

training acc at epoch 1: acc=0.908900
training acc at epoch 2: acc=0.942300
training acc at epoch 3: acc=0.952800
training acc at epoch 4: acc=0.963200
training acc at epoch 5: acc=0.965500
training acc at epoch 6: acc=0.972600
training acc at epoch 7: acc=0.972600
training acc at epoch 8: acc=0.974200
training acc at epoch 9: acc=0.973600
training acc at epoch 10: acc=0.973800
training acc at epoch 11: acc=0.977300
training acc at epoch 12: acc=0.981300
training acc at epoch 13: acc=0.979500
training acc at epoch 14: acc=0.981900
training acc at epoch 15: acc=0.979400
training acc at epoch 16: acc=0.981700
training acc at epoch 17: acc=0.983000
training acc at epoch 18: acc=0.983200
training acc at epoch 19: acc=0.982300
training acc at epoch 20: acc=0.984300
training acc at epoch 21: acc=0.982500
training acc at epoch 22: acc=0.985300
training acc at epoch 23: acc=0.984700
training acc at epoch 24: acc=0.983600
training acc at epoch 25: acc=0.983700
training acc at epoch 26: acc=0.98

In [24]:
LeNet = CNN_Model('LeNet')
LeNet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(LeNet, 0.988, 0.01, 50)

training acc at epoch 1: acc=0.939500
training acc at epoch 2: acc=0.961100
training acc at epoch 3: acc=0.969100
training acc at epoch 4: acc=0.974300
training acc at epoch 5: acc=0.974200
training acc at epoch 6: acc=0.979700
training acc at epoch 7: acc=0.980800
training acc at epoch 8: acc=0.978600
training acc at epoch 9: acc=0.982100
training acc at epoch 10: acc=0.980100
training acc at epoch 11: acc=0.985200
training acc at epoch 12: acc=0.983100
training acc at epoch 13: acc=0.985800
training acc at epoch 14: acc=0.985100
training acc at epoch 15: acc=0.986300
training acc at epoch 16: acc=0.983800
training acc at epoch 17: acc=0.984500
training acc at epoch 18: acc=0.986400
training acc at epoch 19: acc=0.987300
training acc at epoch 20: acc=0.986900
training acc at epoch 21: acc=0.985700
training acc at epoch 22: acc=0.986000
training acc at epoch 23: acc=0.985500
training acc at epoch 24: acc=0.987400
training acc at epoch 25: acc=0.987600
training acc at epoch 26: acc=0.98

In [25]:
LeNet = CNN_Model('LeNet')
LeNet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(LeNet, 0.988, 0.01, 10)

training acc at epoch 1: acc=0.970800
training acc at epoch 2: acc=0.982400
training acc at epoch 3: acc=0.983200
training acc at epoch 4: acc=0.984900
training acc at epoch 5: acc=0.987100
training acc at epoch 6: acc=0.987600
training acc at epoch 7: acc=0.985600
training acc at epoch 8: acc=0.987800
training acc at epoch 9: acc=0.985500
training acc at epoch 10: acc=0.987200
training acc at epoch 11: acc=0.989600


In [26]:
LeNet = CNN_Model('LeNet')
LeNet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(LeNet, 0.988, 0.02, 200)

training acc at epoch 1: acc=0.915500
training acc at epoch 2: acc=0.937600
training acc at epoch 3: acc=0.956700
training acc at epoch 4: acc=0.962300
training acc at epoch 5: acc=0.964300
training acc at epoch 6: acc=0.968900
training acc at epoch 7: acc=0.972600
training acc at epoch 8: acc=0.974800
training acc at epoch 9: acc=0.977200
training acc at epoch 10: acc=0.978300
training acc at epoch 11: acc=0.981500
training acc at epoch 12: acc=0.978500
training acc at epoch 13: acc=0.980900
training acc at epoch 14: acc=0.980800
training acc at epoch 15: acc=0.981300
training acc at epoch 16: acc=0.982300
training acc at epoch 17: acc=0.983000
training acc at epoch 18: acc=0.981500
training acc at epoch 19: acc=0.982100
training acc at epoch 20: acc=0.984400
training acc at epoch 21: acc=0.984600
training acc at epoch 22: acc=0.984600
training acc at epoch 23: acc=0.983300
training acc at epoch 24: acc=0.983000
training acc at epoch 25: acc=0.982800
training acc at epoch 26: acc=0.98

In [27]:
LeNet = CNN_Model('LeNet')
LeNet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(LeNet, 0.988, 0.02, 100)

training acc at epoch 1: acc=0.937100
training acc at epoch 2: acc=0.952800
training acc at epoch 3: acc=0.967000
training acc at epoch 4: acc=0.975800
training acc at epoch 5: acc=0.978300
training acc at epoch 6: acc=0.978000
training acc at epoch 7: acc=0.977200
training acc at epoch 8: acc=0.981300
training acc at epoch 9: acc=0.981800
training acc at epoch 10: acc=0.982700
training acc at epoch 11: acc=0.985800
training acc at epoch 12: acc=0.984000
training acc at epoch 13: acc=0.985500
training acc at epoch 14: acc=0.984600
training acc at epoch 15: acc=0.985800
training acc at epoch 16: acc=0.985400
training acc at epoch 17: acc=0.983700
training acc at epoch 18: acc=0.984700
training acc at epoch 19: acc=0.986500
training acc at epoch 20: acc=0.985600
training acc at epoch 21: acc=0.986400
training acc at epoch 22: acc=0.983200
training acc at epoch 23: acc=0.985100
training acc at epoch 24: acc=0.985400
training acc at epoch 25: acc=0.987000
training acc at epoch 26: acc=0.98

In [28]:
LeNet = CNN_Model('LeNet')
LeNet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(LeNet, 0.988, 0.02, 50)

training acc at epoch 1: acc=0.960500
training acc at epoch 2: acc=0.964100
training acc at epoch 3: acc=0.974900
training acc at epoch 4: acc=0.979200
training acc at epoch 5: acc=0.983600
training acc at epoch 6: acc=0.982000
training acc at epoch 7: acc=0.984400
training acc at epoch 8: acc=0.985800
training acc at epoch 9: acc=0.984400
training acc at epoch 10: acc=0.985400
training acc at epoch 11: acc=0.986700
training acc at epoch 12: acc=0.986600
training acc at epoch 13: acc=0.983800
training acc at epoch 14: acc=0.987800
training acc at epoch 15: acc=0.987500
training acc at epoch 16: acc=0.987900
training acc at epoch 17: acc=0.987600
training acc at epoch 18: acc=0.988000


In [29]:
LeNet = CNN_Model('LeNet')
LeNet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(LeNet, 0.988, 0.02, 10)

training acc at epoch 1: acc=0.975100
training acc at epoch 2: acc=0.975700
training acc at epoch 3: acc=0.983300
training acc at epoch 4: acc=0.989000


In [30]:
LeNet = CNN_Model('LeNet')
LeNet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(LeNet, 0.988, 0.04, 200)

training acc at epoch 1: acc=0.940900
training acc at epoch 2: acc=0.960800
training acc at epoch 3: acc=0.968100
training acc at epoch 4: acc=0.976800
training acc at epoch 5: acc=0.979100
training acc at epoch 6: acc=0.979100
training acc at epoch 7: acc=0.982200
training acc at epoch 8: acc=0.981600
training acc at epoch 9: acc=0.982800
training acc at epoch 10: acc=0.980000
training acc at epoch 11: acc=0.984700
training acc at epoch 12: acc=0.984900
training acc at epoch 13: acc=0.982100
training acc at epoch 14: acc=0.985400
training acc at epoch 15: acc=0.985800
training acc at epoch 16: acc=0.985300
training acc at epoch 17: acc=0.982500
training acc at epoch 18: acc=0.986200
training acc at epoch 19: acc=0.985700
training acc at epoch 20: acc=0.987200
training acc at epoch 21: acc=0.986400
training acc at epoch 22: acc=0.986000
training acc at epoch 23: acc=0.985200
training acc at epoch 24: acc=0.986400
training acc at epoch 25: acc=0.987800
training acc at epoch 26: acc=0.98

In [31]:
LeNet = CNN_Model('LeNet')
LeNet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(LeNet, 0.988, 0.04, 100)

training acc at epoch 1: acc=0.962200
training acc at epoch 2: acc=0.973200
training acc at epoch 3: acc=0.981200
training acc at epoch 4: acc=0.982000
training acc at epoch 5: acc=0.982900
training acc at epoch 6: acc=0.981600
training acc at epoch 7: acc=0.986700
training acc at epoch 8: acc=0.984100
training acc at epoch 9: acc=0.987000
training acc at epoch 10: acc=0.983400
training acc at epoch 11: acc=0.984800
training acc at epoch 12: acc=0.986600
training acc at epoch 13: acc=0.988300


In [32]:
LeNet = CNN_Model('LeNet')
LeNet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(LeNet, 0.988, 0.04, 50)

training acc at epoch 1: acc=0.969700
training acc at epoch 2: acc=0.977900
training acc at epoch 3: acc=0.983000
training acc at epoch 4: acc=0.984200
training acc at epoch 5: acc=0.986800
training acc at epoch 6: acc=0.987900
training acc at epoch 7: acc=0.985500
training acc at epoch 8: acc=0.987400
training acc at epoch 9: acc=0.987400
training acc at epoch 10: acc=0.986000
training acc at epoch 11: acc=0.988800


In [33]:
LeNet = CNN_Model('LeNet')
LeNet.load_parameters("LeNet.params",ctx=ctx)
train_till_acc(LeNet, 0.988, 0.04, 10)

training acc at epoch 1: acc=0.981300
training acc at epoch 2: acc=0.981000
training acc at epoch 3: acc=0.987900
training acc at epoch 4: acc=0.985300
training acc at epoch 5: acc=0.987000
training acc at epoch 6: acc=0.989800
